In [1]:
# built-in
import os
import json

# third-party
import numpy as np
from plotly import graph_objs as go

# local
from files import load_data, save_results
from processing import preprocess, flow_reversal, time_compute, evaluate_extremums, compute_snr

In [2]:
f = open(os.path.join('Aquisicao', 'Cali_factors.json'))
data = json.load(f)
f.close()
id_participants = data.keys()
id_participants

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [3]:
def write_results(scientisst_data, biopac_data, bitalino_data, activities_info, show_fig):
    
    participant_results = {}
    
    for activity in activities_info.keys():

        if show_fig:
            fig = go.Figure()

        a = -scientisst_data['RESP'][activities_info[activity]['start_ind_scientisst']: activities_info[activity]['start_ind_scientisst']+activities_info[activity]['length']]
        b = biopac_data['airflow'][activities_info[activity]['start_ind_biopac']: activities_info[activity]['start_ind_biopac']+activities_info[activity]['length']]
        c = bitalino_data['PZT'][activities_info[activity]['start_ind_bitalino']: activities_info[activity]['start_ind_bitalino']+activities_info[activity]['length']]

        N = len(b)
        time_x = np.linspace(0, N, N)

        a, b, c, integral = preprocess(a, b, c)

        #plot
        if show_fig:
            fig.add_trace(go.Scatter(y = a * 10, mode = 'lines', name='ScientISST'))
            fig.add_trace(go.Scatter(y = integral, mode = 'lines', name='BIOPAC_integral')) 
            fig.add_trace(go.Scatter(y = b, mode = 'lines', name='BIOPAC')) 
            fig.add_trace(go.Scatter(y = c, mode = 'lines', name='BITalino'))

        peaks_a, valleys_a = flow_reversal(a)
        peaks_b, valleys_b = flow_reversal(integral)
        peaks_c, valleys_c = flow_reversal(c)

        tb_b, ti_b , te_b, interval_b, ds_b = time_compute(peaks_b, valleys_b)
        br_b = (60 * len(tb_b)) / np.sum(tb_b)

        FP_s_e, TP_s_e, FN_s_e, performance_clf_s_e, positives_s_e, delay_s_e = evaluate_extremums(peaks_a, peaks_b, tb_b, interval_b)
        FP_s_i, TP_s_i, FN_s_i, performance_clf_s_i, _, delay_s_i = evaluate_extremums(valleys_a, valleys_b, tb_b, interval_b)
        FP_c_e, TP_c_e, FN_c_e, _, _, delay_c_e = evaluate_extremums(peaks_c, peaks_b, tb_b, interval_b)
        FP_c_i, TP_c_i, FN_c_i, _, _, delay_c_i = evaluate_extremums(valleys_c, valleys_b, tb_b, interval_b)


        tb_a, ti_a , te_a, _, ds_a = time_compute(peaks_a, valleys_a)
        tb_c, ti_c , te_c, _, ds_c = time_compute(peaks_c, valleys_c)


        print('Scientisst: tb', tb_a, 'ti' ,ti_a ,'te', te_a)
        br_a = (60*len(tb_a))/np.sum(tb_a)
        br_c = (60*len(tb_c))/np.sum(tb_c)
        brv_a = (np.std(tb_a) / np.mean(tb_a)) * 100
        brv_b = (np.std(tb_b) / np.mean(tb_b)) * 100
        brv_c = (np.std(tb_c) / np.mean(tb_c)) * 100

        if show_fig:
            for i in peaks_a: 
                name = performance_clf_s_e[i]['clf']
                fig.add_vline(x=time_x[i], line_width=1, line_dash="dash", line_color="blue", name="", annotation_text=name)


            for i in valleys_a: 
                name = performance_clf_s_i[i]['clf']
                fig.add_vline(x=time_x[i], line_width=1, line_dash="dash", line_color="green", name="", annotation_text=name)


            for j in peaks_b: 
                if j not in positives_s_e.keys():
                    name = 'FN'
                else: 
                    name = ''
                fig.add_vline(x=time_x[j], line_width=1, line_dash="dash", line_color="black", name="", annotation_text=name)

        
        #flow_reversal
        #ratio_bitalino = (len(ti_c)+len(te_c))/(len(ti_b)+len(te_b))
        print((len(ti_a)+len(te_a))/(len(ti_b)+len(te_b)))
        if show_fig:
            fig.update_layout(title_text = activity)
            fig.show() 

        #save results:
        participant_results_activity = {
            'ScientISST':{
                'peaks': peaks_a.tolist(),
                'valleys': valleys_a.tolist(),
                'amplitude peaks': np.array(a)[peaks_a],
                'amplitude valleys': np.array(a)[valleys_a],
                'SNR': compute_snr(a),
                'TP_i': TP_s_i,
                'FP_i': FP_s_i,
                'FN_i': FN_s_i,
                'TP_e': TP_s_e,
                'FP_e': FP_s_e,
                'FN_e': FN_s_e,
                'tI (s)': ti_a,
                'tE (s)': te_a,
                'tB (s)': tb_a, 
                'delay_i': delay_s_i,
                'delay_e': delay_s_e,
                'ratio': (len(peaks_a) + len(valleys_a)) / (len(peaks_b) + len(valleys_b)),
                'ds (%)': ds_a,
                'BR (bpm)': br_a,
                'BRV (%)' : brv_a,
            },
            'BITalino':{
                'peaks': peaks_c.tolist(),
                'valleys': valleys_c.tolist(),
                'amplitude peaks':np.array(c)[peaks_c],
                'amplitude valleys':np.array(c)[valleys_c],
                'SNR': compute_snr(c),
                'TP_i': TP_c_i,
                'FP_i': FP_c_i,
                'FN_i': FN_c_i,
                'TP_e': TP_c_e,
                'FP_e': FP_c_e,
                'FN_e': FN_c_e,
                'delay_i': delay_c_i,
                'delay_e': delay_c_e,
                'tI (s)': ti_c,
                'tE (s)': te_c,
                'tB (s)': tb_c, 
                'ratio': (len(peaks_c) + len(valleys_c)) / (len(peaks_b) + len(valleys_b)),
                'BR (bpm)': br_c,
                'BRV (%)' : brv_c,
                'ds (%)': ds_c,
            },
            'BIOPAC':{
                'peaks_biopac': peaks_b.tolist(),
                'valleys_biopac': valleys_b.tolist(),
                'amplitude peaks':np.array(b)[peaks_b],
                'amplitude valleys':np.array(b)[valleys_b],
                'SNR': compute_snr(b),
                'tI (s)': ti_b.tolist(),
                'tE (s)': te_b.tolist(),
                'tB (s)': tb_b, 
                'BRV (%)' : brv_b,
                'ds (%)': ds_b,
                'BR (bpm)': br_b,
            }
        }


        participant_results[activity] = participant_results_activity

    save_results(participant_results)






In [4]:
for id in id_participants:
    print('---------',id,'---------------')
    folderpath = os.path.join('Aquisicao', id)
    scientisst_data, biopac_data, bitalino_data, activities_info = load_data('Aquisicao', id)
    write_results(scientisst_data, biopac_data, bitalino_data, activities_info, show_fig=False)

--------- 7OYX ---------------
Scientisst: tb [5.17 5.83 6.69 3.22 2.52 5.04 4.12 4.31 4.28 4.29 4.85 3.63] ti [2.37 3.9  2.69 2.03 0.71 2.68 2.12 1.94 2.08 2.1  2.27 1.98 2.61 3.22] te [1.85 2.49 2.8  1.93 4.   1.19 1.81 2.36 2.   2.37 2.2  2.19 2.58 1.65]
1.0
Scientisst: tb [10.08  7.04 11.05 11.47] ti [3.36 3.24 6.21 5.15 5.32 4.17] te [4.91 6.49 6.72 3.8  4.84 6.32]
1.0
Scientisst: tb [3.75 3.45 3.36 3.24 5.37 5.3  3.3  2.97 3.08 3.22] ti [1.81 1.74 1.69 1.44 1.39 3.75 1.64 1.36 1.45 1.45 1.52] te [1.73 2.1  1.94 1.71 1.67 1.8  3.98 1.55 1.66 1.61 1.63 1.77]
1.0
Scientisst: tb [3.63 3.42 3.17 3.4  3.43 3.43 3.13 3.51 3.46 3.23 3.12 3.64 3.69 3.53
 3.13 4.46 4.03 2.62] ti [1.5  1.34 1.29 1.35 1.42 1.51 1.33 1.52 1.58 1.37 1.18 1.49 1.43 1.4
 1.5  2.18 1.56 1.36] te [2.13 2.08 1.88 2.05 2.01 1.92 1.8  1.99 1.88 1.86 1.94 2.15 2.26 2.13
 1.63 2.28 2.47 1.26]
1.0
Scientisst: tb [3.93 6.15 5.09 5.59 5.75 6.41 5.31 7.1  8.57 7.37] ti [2.07 3.76 2.74 3.07 3.73 3.45 2.87 4.53 3.91 2.63] te

TypeError: save_results() missing 1 required positional argument: 'participant_results'